# 数据加载、存储与文件格式

In [ ]:
!type cars.csv  #读入文件看一下

In [7]:
!type ttstt.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [9]:
import pandas as pd
import numpy as np
s=pd.read_csv('ttstt.csv',header=None,names=['a','b','c','d','message'])
s

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
s.set_index(['message','a'])

,,b,c,d
message,a,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [11]:
s.to_csv('out.csv')   #将结果输出到表格

In [12]:
s.to_csv('out.txt',sep=':')   #将结果输出到txt文件

In [15]:
u=pd.read_table('out.txt',sep=':',index_col=0)  #默认是没有index_col的
u

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [17]:
s.to_csv('out1.txt',sep=' ',index=False,header=False)  #不输出header和index

In [22]:
up1=pd.read_table('out1.txt',sep='\,+',header=None,names=['a','b','c','d','message'])
up1    #不定逗号或者不定数量其他字符分隔的文件读入

C:\Users\lenovo\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## XML和HTML：web信息收集

In [23]:
from lxml.html import parse
from urllib2 import urlopen

In [25]:
parsed=parse(urlopen('http://finance.yahoo.com/quote/AAPL/options?ltr=1'))
doc=parsed.getroot()

In [26]:
links=doc.findall('.//a')
links[15:20]

[<Element a at 0xba3ab88>,
 <Element a at 0xba3abd8>,
 <Element a at 0xba3ac28>,
 <Element a at 0xba3ac78>,
 <Element a at 0xba3acc8>]

In [ ]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

In [ ]:
data = resp.json()
data[0]['title']

In [ ]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

In [27]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [31]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

# 数据规整化

### 轴向连接、数据合并、相加

In [33]:
arr=np.arange(12).reshape(3,4)
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [74]:
df1=pd.DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],columns=['one','two'])
df2=pd.DataFrame(np.arange(4).reshape(2,2),index=['a','c'],columns=['three','four'])
pd.concat([df1,df2],axis=1,keys=['level1','level2'])

level1     level2     
     one two  three four
a      0   1    0.0  1.0
b      2   3    NaN  NaN
c      4   5    2.0  3.0

In [38]:
pd.concat([df1,df2],ignore_index=True)  #可以忽略无意义的行索引

,four,one,three,two
0,NaN,0.0,NaN,1.0
1,NaN,2.0,NaN,3.0
2,NaN,4.0,NaN,5.0
3,1.0,NaN,0.0,NaN
4,3.0,NaN,2.0,NaN


In [75]:
df1.columns=df2.columns
df1

,three,four
a,0,1
b,2,3
c,4,5


In [44]:
pd.concat([df1,df2],ignore_index=True)

,three,four
0,0,1
1,2,3
2,4,5
3,0,1
4,2,3


In [48]:
pd.merge(df1,df2,how='outer',left_index=True,right_index=True,suffixes=['_df1','_df2'])  
#merge只能合并相同的行，可合并多个index的df. pd.concat([df1,df2],axis=1)也可以合并多个index的df, 但不会改变合并后相同的列的名称

,three_df1,four_df1,three_df2,four_df2
a,0,1,0.0,1.0
b,2,3,NaN,NaN
c,4,5,2.0,3.0


In [81]:
df3=pd.concat([df1,df2],axis=1)
df3

,three,four,three,four
a,0,1,0.0,1.0
b,2,3,NaN,NaN
c,4,5,2.0,3.0


In [83]:
df1.add(df2,fill_value=0)    #add则是对行和列都进行了合并 并将对应结果相加

,three,four
a,0.0,2.0
b,2.0,3.0
c,6.0,8.0


#### 合并重叠数据

In [84]:
df3=pd.DataFrame({'a':[1,np.nan,5,np.nan],
                 'b':[np.nan,2,np.nan,6],
                 'c':range(2,18,4)})
df4=pd.DataFrame({'a':[5,4,np.nan,3,7],
                 'b':[np.nan,3,4,6,8]})
df3.combine_first(df4)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 重塑和轴向旋转

In [87]:
df1.index.name='word'
df1.columns.name='ranks'
result=df1.stack()    #重塑
result

word  ranks
a     three    0
      four     1
b     three    2
      four     3
c     three    4
      four     5
dtype: int32

In [89]:
result.unstack()

ranks,three,four
word,,
a,0,1
b,2,3
c,4,5


In [91]:
result.unstack('word')

word,a,b,c
ranks,,,
three,0,2,4
four,1,3,5


# 数据转换（大小写变换）

In [8]:
import pandas as pd
import numpy as np
data=pd.DataFrame({'food':['bacon','pulled pork','bacon','Pastrami','corned beef','Bacon',
                        'pastrami','honey ham','nova lox'],
                  'ounces':[4,3,12,6,7.5,8,3,5,6]})
meat_to_animal={'bacon':'pig','pulled pork':'pig','pastrami':'cow','corned beef':'cow','honey ham':'pig',
               'nova lox':'salmon'}
data['animal']=data['food'].map(str.lower).map(meat_to_animal) 
# map可以接受一个函数或含有映射关系的字典型对象
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [9]:
data['animal1']=data['food'].map(lambda x:meat_to_animal[x.lower()])
data

,food,ounces,animal,animal1
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,Pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,Bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


In [10]:
data1=data.drop('animal1',axis=1)
data1

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [11]:
data2=data.copy()
data2['food']=data2['food'].map(str.lower)
data2

,food,ounces,animal,animal1
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


In [12]:
data2.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
dtype: bool

In [13]:
data2.drop_duplicates()

,food,ounces,animal,animal1
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


In [20]:
data2.drop_duplicates(['food'],keep='last')  #只能对行操作

,food,ounces,animal,animal1
1,pulled pork,3.0,pig,pig
4,corned beef,7.5,cow,cow
5,bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


In [19]:
data2.drop_duplicates?

In [24]:
data2.replace(['bacon','pig'],[np.nan,0])

,food,ounces,animal,animal1
0,NaN,4.0,0,0
1,pulled pork,3.0,0,0
2,NaN,12.0,0,0
3,pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,NaN,8.0,0,0
6,pastrami,3.0,cow,cow
7,honey ham,5.0,0,0
8,nova lox,6.0,salmon,salmon


In [28]:
data2.columns=data2.columns.map(str.upper)
data2

,FOOD,OUNCES,ANIMAL,ANIMAL1
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


In [29]:
data2.rename(columns={'ANIMAL1':'animal_plus'})

,FOOD,OUNCES,ANIMAL,animal_plus
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


In [32]:
bins=[0,5,10,15]
cats=pd.cut(data2['OUNCES'],bins,right=False,labels=['cheap','modest','expensive'])

In [33]:
cats

0        cheap
1        cheap
2    expensive
3       modest
4       modest
5       modest
6        cheap
7       modest
8       modest
Name: OUNCES, dtype: category
Categories (3, object): [cheap < modest < expensive]

In [41]:
data3=np.random.uniform(0,10,size=20)
#pd.cut(data3,4,precision=2,labels=['little','middle','large','huge'])
cat1=pd.cut(data3,4,precision=2)   #pd.cut是等长划分
cat1

[(7.3, 9.65], (2.58, 4.94], (0.22, 2.58], (2.58, 4.94], (2.58, 4.94], ..., (4.94, 7.3], (2.58, 4.94], (2.58, 4.94], (2.58, 4.94], (0.22, 2.58]]
Length: 20
Categories (4, interval[float64]): [(0.22, 2.58] < (2.58, 4.94] < (4.94, 7.3] < (7.3, 9.65]]

In [42]:
cat2=pd.qcut(data3,4)   #等数量的划分
cat2

[(6.205, 9.651], (1.045, 3.256], (0.227, 1.045], (1.045, 3.256], (3.256, 6.205], ..., (3.256, 6.205], (3.256, 6.205], (3.256, 6.205], (3.256, 6.205], (1.045, 3.256]]
Length: 20
Categories (4, interval[float64]): [(0.227, 1.045] < (1.045, 3.256] < (3.256, 6.205] < (6.205, 9.651]]

In [43]:
cat1.value_counts()

(0.22, 2.58]    7
(2.58, 4.94]    7
(4.94, 7.3]     2
(7.3, 9.65]     4
dtype: int64

In [44]:
cat2.value_counts()

(0.227, 1.045]    5
(1.045, 3.256]    5
(3.256, 6.205]    5
(6.205, 9.651]    5
dtype: int64

In [46]:
pd.qcut(data3,[0,0.1,0.5,0.9,1])  #设置自定义的分位数，包括端点

[(3.256, 8.38], (0.478, 3.256], (0.478, 3.256], (0.478, 3.256], (3.256, 8.38], ..., (3.256, 8.38], (3.256, 8.38], (3.256, 8.38], (3.256, 8.38], (0.478, 3.256]]
Length: 20
Categories (4, interval[float64]): [(0.227, 0.478] < (0.478, 3.256] < (3.256, 8.38] < (8.38, 9.651]]

In [52]:
data3.sort()
data3

array([ 0.22802051,  0.22842043,  0.50531495,  0.71212235,  0.80383416,
        1.12603106,  2.40557267,  2.60733428,  2.90487694,  2.94611655,
        3.56525909,  3.89066915,  4.14318276,  4.33042257,  5.96287528,
        6.93085737,  8.02611415,  8.3202633 ,  8.91291823,  9.65104331])

In [50]:
data3.argmin()

13

In [53]:
import random
np.random.seed(12345)

In [54]:
data=pd.DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [55]:
data[(np.abs(data)>3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [58]:
col=data[2]
col[np.abs(col)>3]

5      3.248944
102    3.176873
324    3.260383
499   -3.056990
586   -3.184377
Name: 2, dtype: float64

In [59]:
data[np.abs(data)>3]=np.sign(data)*3  #np.sign取的是符号
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


#### 随机取DataFrame的行子集

In [110]:
df=pd.DataFrame(np.arange(20).reshape(10,2))
df1=df.take(np.random.permutation(len(df))[:4])
df1

,0,1
8,16,17
4,8,9
9,18,19
1,2,3


### 将分类变量转换成指标矩阵

In [115]:
#手动做法
df=pd.DataFrame({'key':['b','b','a','c','a','b'],
                 'data1':range(6),
                'labels':[1]*6})
df

,data1,key,labels
0,0,b,1
1,1,b,1
2,2,a,1
3,3,c,1
4,4,a,1
5,5,b,1


In [102]:
s=pd.DataFrame()   #创建空的dataframe
for i in range(len(df)):
    u=df.iloc[i]
    df1=pd.Series(u['labels'],index=[u['key']])
    s=pd.concat([s,df1],axis=1)
p=s.T.fillna(0)
p['list']=range(6)
p.set_index('list')

,a,b,c
list,,,
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [113]:
pd.get_dummies(df['key'])   #调用函数计算指标矩阵的办法

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [118]:
dummies=pd.get_dummies(df['key'],prefix='key')
df_with_dummy=df[['data1']].join(dummies)   #df[['data1']] 此处有两个括号,两个括号的结果还是dataframe
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [122]:
mnames=['movie_id','title','genres']
movies=pd.read_table('pydata-book-2nd-edition/datasets/movielens/movies.dat',sep='::',header=None,names=mnames)

C:\Users\lenovo\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [123]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [132]:
genre_iter=(set(x.split('|')) for x in movies.genres)
genres=sorted(set.union(*genre_iter))    # *表示取genre_iter这个元组中的每一个set, 并将其合并到一个新的set(set.union)
dummies=pd.DataFrame(np.zeros((len(movies),len(genres))),columns=genres)
dummies.columns

Index([u'Action', u'Adventure', u'Animation', u'Children's', u'Comedy',
       u'Crime', u'Documentary', u'Drama', u'Fantasy', u'Film-Noir', u'Horror',
       u'Musical', u'Mystery', u'Romance', u'Sci-Fi', u'Thriller', u'War',
       u'Western'],
      dtype='object')

In [133]:
for i,gen in enumerate(movies.genres):
    dummies.iloc[i][gen.split('|')]=1

In [135]:
movies_windic=movies.join(dummies.add_prefix('Genre_'))
movies_windic.head()

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Genre_Fantasy,Genre_Film-Noir,Genre_Horror,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
gen=movies.genres[0]
pd.get_dummies(pd.Series(gen.split('|')))

,Animation,Children's,Comedy
0,1,0,0
1,0,1,0
2,0,0,1
